In [104]:
def is_open(site_id, time):
    "Check if site is open"
    site = df_timetables[df_timetables.site_id == site_id]
    for index, row in df_timetables.iterrows():
        site_opening_datetime = row['opening_datetime']
        site_closing_datetime = row['closing_datetime']
#         print(site_opening_datetime, ':', site_closing_datetime)
#         print((time > site_opening_datetime) & (time < site_closing_datetime))
    return (time > site_opening_datetime) & (time < site_closing_datetime)
    
is_open(235, pd.to_datetime("today"))

False

In [106]:
#!/usr/bin/env python3
import pandas as pd

# load data
fp_data = '/home/amait/Téléchargements/Test_technique_data_engineer_Affluences/data.csv'
data = pd.read_csv(fp_data, delimiter=',')
fp_time = '/home/amait/Téléchargements/Test_technique_data_engineer_Affluences/timetables.csv'
timetables = pd.read_csv(fp_time, delimiter=',')

# copy
df_data = data.copy()
df_timetables = timetables.copy()

# to_datetime
df_data.last_record_datetime = df_data.last_record_datetime.apply(pd.to_datetime)
df_timetables.opening_datetime = df_timetables.opening_datetime.apply(pd.to_datetime)
df_timetables.closing_datetime = df_timetables.closing_datetime.apply(pd.to_datetime)

# add delta column
time = pd.to_datetime("today")
df_data['last_record_datetime2'] = time - df_data.last_record_datetime

# add alert column
df_data['alert'] = time - df_data.last_record_datetime
mask0 = df_data.last_record_datetime2 < pd.to_timedelta("0 days 02:00:00.00000")
mask1 = (df_data.last_record_datetime2 >= pd.to_timedelta("0 days 02:00:00.00000")) & (df_data.last_record_datetime2 < pd.to_timedelta("1 days 00:00:00.00000"))
mask2 = (df_data.last_record_datetime2 >= pd.to_timedelta("1 days 00:00:00.00000")) & (df_data.last_record_datetime2 < pd.to_timedelta("2 days 00:00:00.00000"))
mask3 = df_data.last_record_datetime2 > pd.to_timedelta("2 days 00:00:00.00000")
df_data['alert'].loc[mask0] = 0
df_data['alert'].loc[mask1] = 1
df_data['alert'].loc[mask2] = 2
df_data['alert'].loc[mask3] = 3

# delete row without alert
df_data = df_data[df_data.alert > 0]


def is_open(site_id, time):
    "Check if site is open"
    site = df_timetables[df_timetables.site_id == site_id]
    for index, row in df_timetables.iterrows():
        site_opening_datetime = row['opening_datetime']
        site_closing_datetime = row['closing_datetime']
#         print(site_opening_datetime, ':', site_closing_datetime)
#         print((time > site_opening_datetime) & (time < site_closing_datetime))
    return (time > site_opening_datetime) & (time < site_closing_datetime)

    
for index, row in df_data.iterrows():
    site_id = row['site_id']
    ok = is_open(site_id, time)
    if is_open(site_id, time):
        print(f"Sensor {row['sensor_name']} with identifier {row['sensor_identifier']} triggers an alert at {time} with level {row['alert']} with last data recorded at {row['last_record_datetime']}")

/home/amait/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [77]:
df_data.head()

,sensor_identifier,sensor_name,site_id,last_record_datetime,last_record_datetime2,alert
0,3,sensor_3,388,2019-01-30 13:55:00,826 days 05:13:52.287056,3
1,4,sensor_4,389,2019-01-30 14:00:00,826 days 05:08:52.287056,3
2,7,sensor_7,1784,2021-03-25 06:11:00,41 days 12:57:52.287056,3
3,8,sensor_8,1784,2021-05-04 15:17:00,1 days 03:51:52.287056,2
4,9,sensor_9,443,2021-05-04 15:15:00,1 days 03:53:52.287056,2


In [88]:
df_timetables.head()

,site_id,opening_datetime,closing_datetime
0,107,2021-05-05 10:00:00,2021-05-05 18:00:00
1,132,2021-05-05 07:00:00,2021-05-05 19:00:00
2,233,2021-05-05 00:00:00,2021-05-05 23:55:00
3,235,2021-05-05 00:00:00,2021-05-05 23:55:00
4,272,2021-05-05 08:30:00,2021-05-05 18:00:00
